In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

In [ ]:
from transformers import BartModel, PreTrainedTokenizerFast, BartForConditionalGeneration
import torch

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization') # TOKENIZER : 컴퓨터가 자연어를 이해할 수 있도록
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization') # SUMMARIZATION : 요약해주는 모델 (지금 모델)

text = "과거를 떠올려보자. 방송을 보던 우리의 모습을. 독보적인 매체는 TV였다. 온 가족이 둘러앉아 TV를 봤다. 간혹 가족들끼리 뉴스와 드라마, 예능 프로그램을 둘러싸고 리모컨 쟁탈전이 벌어지기도  했다. 각자 선호하는 프로그램을 ‘본방’으로 보기 위한 싸움이었다. TV가 한 대인지 두 대인지 여부도 그래서 중요했다. 지금은 어떤가. ‘안방극장’이라는 말은 옛말이 됐다. TV가 없는 집도 많다. 미디어의 혜 택을 누릴 수 있는 방법은 늘어났다. 각자의 방에서 각자의 휴대폰으로, 노트북으로, 태블릿으로 콘텐츠 를 즐긴다."
print(f'입력 문서: {text}')

raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]))
print(f'요약 결과: {tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)}')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

입력 문서: 과거를 떠올려보자. 방송을 보던 우리의 모습을. 독보적인 매체는 TV였다. 온 가족이 둘러앉아 TV를 봤다. 간혹 가족들끼리 뉴스와 드라마, 예능 프로그램을 둘러싸고 리모컨 쟁탈전이 벌어지기도  했다. 각자 선호하는 프로그램을 ‘본방’으로 보기 위한 싸움이었다. TV가 한 대인지 두 대인지 여부도 그래서 중요했다. 지금은 어떤가. ‘안방극장’이라는 말은 옛말이 됐다. TV가 없는 집도 많다. 미디어의 혜 택을 누릴 수 있는 방법은 늘어났다. 각자의 방에서 각자의 휴대폰으로, 노트북으로, 태블릿으로 콘텐츠 를 즐긴다.


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


요약 결과: TV가 없는 집도 많아지고 미디어의 혜 택을 누릴 수 있는 방법은 늘어났다.


## Fine-tuning 과정

#### 1. 데이터 확인하기

In [ ]:
with open('/content/drive/MyDrive/Data/train.csv', 'r', encoding='utf-8') as f:
    train_dataset = [i.strip() for i in f][1:101]

with open('/content/drive/MyDrive/Data/test.csv', 'r', encoding='utf-8') as f:
    test_dataset = [i.strip() for i in f][1:11]

In [ ]:
len(train_dataset), len(test_dataset)

(100, 10)

In [ ]:
train_dataset[:1]

['REPORT-edit-00002,"윤석열 국민의힘 대선후보는 자기만의 공부법이 있었다.과거 고시공부를 할 때도 다른 고시생들과 공부방법이 달랐다.다른 고시생들은 주로 여러 과목의 책을 한권의 서브노트로 요약하고 이 노트를 달달 외우는 식으로 공부를 했다고 한다.하지만 윤 후보는 그게 아니었다.서브노트를 만들지 않고 모든 과목의 책을 그냥 읽었다.그러다보니2~3시간이 지나면 그가 공부하는 책상에는 책이 수북이 쌓였다.책을 보다 궁금한 부분이 있으면 관련된 책을 모두 찾아 본 것이다.그러니 고시 공부에 진도가 나갈 리가 없었다.윤 후보가9수(修)만에 고시에 합격한 이유를 알 것 같다.']

#### 2. Pytorch 데이터셋 정의

In [ ]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from torch.utils.data import Dataset, DataLoader, IterableDataset

class KoBARTSummaryDataset(Dataset):
    def __init__(self, file, tok, max_len, pad_index=None, ignore_index=-100):
        super().__init__()
        self.tok = tok
        self.max_len = max_len
        self.docs = pd.read_csv(file, sep=',')  # 쉼표로 구분된 파일 읽기
        self.len = self.docs.shape[0]
        if pad_index is None:
            self.pad_index = self.tok.pad_token_id
        else:
            self.pad_index = pad_index
        self.ignore_index = ignore_index

    def add_padding_data(self, inputs):
        if len(inputs) < self.max_len:
            pad = np.array([self.pad_index] * (self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:self.max_len]
        return inputs

    def add_ignored_data(self, inputs):
        if len(inputs) < self.max_len:
            pad = np.array([self.ignore_index] * (self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:self.max_len]
        return inputs

    def __getitem__(self, idx):
        instance = self.docs.iloc[idx]
        input_ids = self.tok.encode(instance['passage'])  # 본문 데이터
        input_ids = self.add_padding_data(input_ids)

        label_ids = self.tok.encode(instance['summary1'])  # 요약 데이터
        label_ids.append(self.tok.eos_token_id)
        dec_input_ids = [self.pad_index]
        dec_input_ids += label_ids[:-1]
        dec_input_ids = self.add_padding_data(dec_input_ids)
        label_ids = self.add_ignored_data(label_ids)

        return {'input_ids': np.array(input_ids, dtype=np.int_),
                'decoder_input_ids': np.array(dec_input_ids, dtype=np.int_),
                'labels': np.array(label_ids, dtype=np.int_)}

    def __len__(self):
        return self.len

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("hyunwoongko/kobart")
model = BartForConditionalGeneration.from_pretrained("hyunwoongko/kobart")

tokenizer_config.json:   0%|          | 0.00/337 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/109 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


pytorch_model.bin:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [ ]:
train_dataset = KoBARTSummaryDataset('/content/drive/MyDrive/Data/train.csv', tokenizer, max_len=256)
test_dataset = KoBARTSummaryDataset('/content/drive/MyDrive/Data/test.csv', tokenizer, max_len=256)

#### 2. 하이퍼 파라미터 수정

In [ ]:
learning_rate = 5e-5
epoch = 10
batch_size = 32

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

#### 3. 모델 학습

In [ ]:
from torch.optim import Adam
from tqdm import tqdm

In [ ]:
parameters = model.parameters()
optimizer = Adam(parameters, lr=learning_rate)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
epoch = epoch
is_cuda = True  # GPU 환경
gradient_accumulation_steps = 4  # Gradient Accumulation 설정
scaler = torch.cuda.amp.GradScaler()  # Mixed Precision Training 사용

if is_cuda:
    model.cuda()

for e in range(epoch):
    model.train()  # 모델을 학습 모드로 설정
    pbar = tqdm(train_dataloader, total=len(train_dataloader))
    optimizer.zero_grad()  # Optimizer 초기화
    for step, batch in enumerate(pbar):
        batch = {k: v.cuda() for k, v in batch.items()} if is_cuda else batch

        with torch.cuda.amp.autocast():  # Mixed Precision 활성화
            output = model(**batch)
            loss = output.loss / gradient_accumulation_steps  # 손실을 누적 단계로 나누기

        scaler.scale(loss).backward()  # Mixed Precision Scaler로 역전파

        # Gradient Accumulation 조건
        if (step + 1) % gradient_accumulation_steps == 0:
            scaler.step(optimizer)  # Optimizer 업데이트
            scaler.update()  # Scaler 업데이트
            optimizer.zero_grad()  # Optimizer 초기화

        pbar.set_description(f'Epoch: {e}, Step: {step}, Loss: {loss.item()}')

    # Epoch 종료 후 GPU 메모리 캐시 정리
    torch.cuda.empty_cache()

<ipython-input-16-029220213936>:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()  # Mixed Precision Training 사용
  0%|          | 0/463 [00:00<?, ?it/s]<ipython-input-16-029220213936>:16: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():  # Mixed Precision 활성화
Epoch: 9, Step: 462, Loss: 0.1894734650850296: 100%|██████████| 463/463 [04:01<00:00,  1.92it/s]


In [ ]:
text = "과거를 떠올려보자. 방송을 보던 우리의 모습을. 독보적인 매체는 TV였다. 온 가족이 둘러앉아 TV를 봤다. 간혹 가족들끼리 뉴스와 드라마, 예능 프로그램을 둘러싸고 리모컨 쟁탈전이 벌어지기도  했다. 각자 선호하는 프로그램을 ‘본방’으로 보기 위한 싸움이었다. TV가 한 대인지 두 대인지 여부도 그래서 중요했다. 지금은 어떤가. ‘안방극장’이라는 말은 옛말이 됐다. TV가 없는 집도 많다. 미디어의 혜택을 누릴 수 있는 방법은 늘어났다. 각자의 방에서 각자의 휴대폰으로, 노트북으로, 태블릿으로 콘텐츠 를 즐긴다."
print(f'입력 문서 : {text}')

raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]).cuda())
print(f'요약 결과 : {tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)}')

입력 문서 : 과거를 떠올려보자. 방송을 보던 우리의 모습을. 독보적인 매체는 TV였다. 온 가족이 둘러앉아 TV를 봤다. 간혹 가족들끼리 뉴스와 드라마, 예능 프로그램을 둘러싸고 리모컨 쟁탈전이 벌어지기도  했다. 각자 선호하는 프로그램을 ‘본방’으로 보기 위한 싸움이었다. TV가 한 대인지 두 대인지 여부도 그래서 중요했다. 지금은 어떤가. ‘안방극장’이라는 말은 옛말이 됐다. TV가 없는 집도 많다. 미디어의 혜택을 누릴 수 있는 방법은 늘어났다. 각자의 방에서 각자의 휴대폰으로, 노트북으로, 태블릿으로 콘텐츠 를 즐긴다.


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


요약 결과 : TV의 혜택을 누릴 수 있는 방법은 늘어났으며 각자의 휴대폰으로 콘텐츠 를 즐긴다.


4. 모델 성능 평가

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=ec331acc13ad6d0a64641d10057bfe3d39145a3db339e8a4ca132e6a8d6107d0
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

reference_summary = "과거를 떠올려보자. 방송을 보던 우리의 모습을. 독보적인 매체는 TV였다. 온 가족이 둘러앉아 TV를 봤다. 간혹 가족들끼리 뉴스와 드라마, 예능 프로그램을 둘러싸고 리모컨 쟁탈전이 벌어지기도  했다. 각자 선호하는 프로그램을 ‘본방’으로 보기 위한 싸움이었다. TV가 한 대인지 두 대인지 여부도 그래서 중요했다. 지금은 어떤가. ‘안방극장’이라는 말은 옛말이 됐다. TV가 없는 집도 많다. 미디어의 혜택을 누릴 수 있는 방법은 늘어났다. 각자의 방에서 각자의 휴대폰으로, 노트북으로, 태블릿으로 콘텐츠 를 즐긴다."
generated_summary = "TV의 혜택을 누릴 수 있는 방법은 늘어났고 각자의 휴대폰으로 콘텐츠 를 즐긴다."

# ROUGE 점수 계산
scores = scorer.score(reference_summary, generated_summary)
print(scores)

{'rouge1': Score(precision=1.0, recall=0.25, fmeasure=0.4), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=1.0, recall=0.25, fmeasure=0.4)}


5. 모델 저장

In [ ]:
# 모델 저장하기
torch.save(model.state_dict(), "/content/drive/MyDrive/Colab Notebooks/Diary_Summary.pt")
print("모델 저장 완료")

모델 저장 완료


In [ ]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast
import torch

# 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("hyunwoongko/kobart")

# 모델 초기화
model = BartForConditionalGeneration.from_pretrained("hyunwoongko/kobart")

# 저장된 가중치 로드
model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/Diary_Summary.pt"))
model.eval()  # 평가 모드 설정


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
<ipython-input-30-3cbd767668c7>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on G

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [ ]:
text = "오늘은 하루 종일 추운 날씨였지만, 햇살이 밝아서 기분이 나쁘지 않았다. 아침에는 일찍 일어나서 커피를 마시며 간단히 스트레칭을 했다. 요즘 몸이 뻐근한 느낌이 들어 조금이라도 몸을 풀어야겠다는 생각이 들었다. 오전에는 작업 중인 프로젝트에 대해 자료를 정리했다. 특히, 최적의 재활용 스테이션 위치를 선정하는 부분에서 데이터를 분석하면서 고민이 많았다. 문제 해결을 위해 새로운 아이디어가 떠올랐는데, 지역 주민들이 재활용을 얼마나 적극적으로 참여할 수 있을지 설문조사를 병행해보는 것도 좋은 방법일 것 같다. 점심에는 동료들과 근처 작은 식당에서 한식 정식을 먹었다. 오랜만에 대화를 나누며 웃을 수 있어서 기분이 한결 가벼워졌다. 오후에는 잠깐 산책도 하며 머리를 식히려 했지만, 바람이 꽤 차가워서 오래 걷지는 못했다. 저녁에는 책 한 권을 읽었다. 요즘 흥미를 가지고 있는 현대 소설인데, 주인공의 감정 변화가 섬세하게 묘사되어 있어서 몰입하기 쉬웠다. 글을 읽다 보니, 내 프로젝트에서도 감정 표현을 어떻게 시각적으로 전달할지 생각해보게 됐다. 오늘 하루를 돌아보니 바쁘지만 나름 알찬 하루였던 것 같다. 내일은 조금 더 계획적으로 시간을 쓰고 싶다. 매 순간을 가치 있게 보내는 것이 올해 목표이니, 더 노력해야겠다."
input_ids = tokenizer.encode(text, return_tensors="pt")
input_ids = torch.cat([torch.tensor([[tokenizer.bos_token_id]]), input_ids, torch.tensor([[tokenizer.eos_token_id]])], dim=-1)

# 모델 추론 (길이 조정)
summary_ids = model.generate(
    input_ids,
    max_length=500,  # 요약 최대 길이
    min_length=100,   # 요약 최소 길이
    length_penalty=1.0,  # 길이 패널티
    num_beams=4,  # 빔 서치 사용
    early_stopping=True  # 조건 만족 시 생성 중단
)
summary_text = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
print("요약 결과:", summary_text)

요약 결과: 작업 재활용 스테이션 위치를 선정하는 부분에서 데이터를 분석하면서 고민이 많았는데 지역 주민들이 재활용을 얼마나 적극적으로 참여할 수 있을지 설문조사를 병행해보는 것도 좋은 방법일 것 같았다.                                                               


6. 모델 - 앱 연결

In [ ]:
from flask import Flask, request, jsonify
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast
import torch

app = Flask(__name__)

# 모델 및 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("hyunwoongko/kobart")
model = BartForConditionalGeneration.from_pretrained("hyunwoongko/kobart")
model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/Diary_Summary.pt"))
model.eval()

@app.route('/summarize', methods=['POST'])
def summarize():
    data = request.json
    text = data.get('text', '')

    if not text:
        return jsonify({'error': 'Text is required'}), 400

    # 입력 데이터 처리
    input_ids = tokenizer.encode(text, return_tensors="pt")
    input_ids = torch.cat([torch.tensor([[tokenizer.bos_token_id]]), input_ids, torch.tensor([[tokenizer.eos_token_id]])], dim=-1)

    # 모델 추론
    summary_ids = model.generate(
        input_ids,
        max_length=100,
        min_length=50,
        length_penalty=1.0,
        num_beams=4,
        early_stopping=True
    )
    summary_text = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)

    return jsonify({'summary': summary_text})

if __name__ == '__main__':
    app.run(debug=True)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
<ipython-input-33-abb928278880>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on G

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
